# CNN with 1D kernels and max-pooling 

In [0]:
# Install TensorFlow
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
tf.__version__

'2.2.0'

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
!apt install fluidsynth
!pip install midi2audio
!pip install pretty_midi

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fluid-soundfont-gm libfluidsynth1 libqt5x11extras5 qsynth
Suggested packages:
  fluid-soundfont-gs timidity jackd
The following NEW packages will be installed:
  fluid-soundfont-gm fluidsynth libfluidsynth1 libqt5x11extras5 qsynth
0 upgraded, 5 newly installed, 0 to remove and 31 not upgraded.
Need to get 120 MB of archives.
After this operation, 150 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fluid-soundfont-gm all 3.1-5.1 [119 MB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libfluidsynth1 amd64 1.1.9-1 [137 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fluidsynth amd64 1.1.9-1 [20.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libqt5x11extras5 amd64 5.9.5-0ubuntu1 [8,596 B]
Get:5 http://archive.ubuntu.com/ubuntu bionic/uni

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Dense, Dropout, Input, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.callbacks import ModelCheckpoint, LambdaCallback
import numpy as np
import pylab as plt

import sys
sys.path.insert(1, '/content/gdrive/My Drive/Music Gen/import/')
import models
import trainFac
import midiUtils

## NN Architecture

In [8]:
class Model_CNN_Maxpool(models.Model_1Dout):

    def __init__(self, input_length):
        super().__init__(input_length)
        self.createModel()

    def createModel(self):
        model = Sequential()
        model.add(Input(shape=(self.input_length, 48, 1)))

        model.add(Conv2D(16, (1, 5), activation='relu'))
        model.add(MaxPool2D(pool_size=(2, 1)))

        model.add(Conv2D(16, (1, 5), activation='relu'))
        model.add(MaxPool2D(pool_size=(2, 1)))

        model.add(Conv2D(32, (5, 5), activation='relu'))

        model.add(Flatten())
        model.add(Dense(48, activation='sigmoid'))
        # save to self
        self.model = model


modelCompound = Model_CNN_Maxpool(120)

# modelCompound.model.load_weights('/content/gdrive/My Drive/Music Gen/networks/cnn_maxpool1D_kernel1D/train1/checkpoints/cnn-20-accuracy0.477.hdf5')
modelCompound.model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 120, 44, 16)       96        
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 60, 44, 16)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 60, 40, 16)        1296      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 30, 40, 16)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 26, 36, 32)        12832     
_________________________________________________________________
flatten_1 (Flatten)          (None, 29952)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 48)               

## Training

In [0]:
paths = trainFac.Paths(
    rootDir = '/content/gdrive/My Drive/Music Gen',
    trainPath = 'train_batches/train_7840_chorus_npz_500',
    testPath  = 'test_batches/test_800_chorus_npz_100',
    testSongPath = 'test_generation/test_songs',
    testGenerateSavePath = 'networks/cnn_maxpool1D_kernel1D/train1/generate',
    checkpointBasePath = 'networks/cnn_maxpool1D_kernel1D/train1/checkpoints',
    checkpointName='cnn-{epoch:02d}-accuracy{val_accuracy:.3f}.hdf5')
# create paths if they dont exist
paths.createSavePaths()

trainFacility = trainFac.TrainFacility(modelCompound, paths)
trainFacility.compile()

trainFacility.modelCompound.model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 120, 44, 16)       96        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 60, 44, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 60, 40, 16)        1296      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 40, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 26, 36, 32)        12832     
_________________________________________________________________
flatten (Flatten)            (None, 29952)             0         
_________________________________________________________________
dense (Dense)                (None, 48)                1

In [0]:
history = trainFacility.fit(epochs=20)

Epoch 1/20
10000/10000 [==============================] - ETA: 0s - loss: 3.5240 - accuracy: 0.3904
Epoch 00001: saving model to /content/gdrive/My Drive/Music Gen/networks/cnn_maxpool1D_kernel1D_cathegorical/train1/checkpoints/cnn-01-accuracy0.431.hdf5
create save path
 -- Generating songs after epoch: 0
 -- Generating songs finished
10000/10000 [==============================] - 411s 41ms/step - loss: 3.5240 - accuracy: 0.3904 - val_loss: 3.2714 - val_accuracy: 0.4310
Epoch 2/20
 9999/10000 [============================>.] - ETA: 0s - loss: 3.0927 - accuracy: 0.4351
Epoch 00002: saving model to /content/gdrive/My Drive/Music Gen/networks/cnn_maxpool1D_kernel1D_cathegorical/train1/checkpoints/cnn-02-accuracy0.400.hdf5
10000/10000 [==============================] - 345s 34ms/step - loss: 3.0928 - accuracy: 0.4350 - val_loss: 3.1478 - val_accuracy: 0.3996
Epoch 3/20
 9999/10000 [============================>.] - ETA: 0s - loss: 3.0061 - accuracy: 0.4419
Epoch 00003: saving model to /con

In [0]:
import json
with open('/content/gdrive/My Drive/Music Gen/networks/cnn_maxpool1D_kernel1D_cathegorical/train1/history.json', 'w') as fp:
    json.dump(history.history, fp, indent=4)

## Experiments

In [0]:
net_name = 'CNN Maxpool'
save_path = '/content/gdrive/My Drive/Music Gen/results/cnn-maxpool/'

In [9]:
song_name = "Can't Stop"
seedPath = '/content/gdrive/My Drive/Music Gen/test_generation/test_songs/cant_stop.npz'

midi = modelCompound.generateMidi(
    seedPath,
    280,
    threshold=0.1)

midiUtils.plotMidi(save_path, net_name, song_name, midi)
midiUtils.playMidi(midi, save_path + midiUtils.joinNames(net_name, song_name))

Output hidden; open in https://colab.research.google.com to view.

In [0]:
song_name = 'Hells Bells'
seedPath = '/content/gdrive/My Drive/Music Gen/test_generation/test_songs/hells_bells.npz'

midi = modelCompound.generateMidi(
    seedPath,
    280,
    threshold=0.1)

midiUtils.plotMidi(save_path, net_name, song_name, midi)
midiUtils.playMidi(midi, save_path + midiUtils.joinNames(net_name, song_name))

Output hidden; open in https://colab.research.google.com to view.

In [0]:
song_name = 'Layla'
seedPath = '/content/gdrive/My Drive/Music Gen/test_generation/test_songs/layla.npz'

midi = modelCompound.generateMidi(
    seedPath,
    280,
    threshold=0.1)

midiUtils.plotMidi(save_path, net_name, song_name, midi)
midiUtils.playMidi(midi, save_path + midiUtils.joinNames(net_name, song_name))

Output hidden; open in https://colab.research.google.com to view.

In [0]:
song_name = 'Nothing Else Matters'
seedPath = '/content/gdrive/My Drive/Music Gen/test_generation/test_songs/nothing_else_matters.npz'

midi = modelCompound.generateMidi(
    seedPath,
    280,
    threshold=0.1)

midiUtils.plotMidi(save_path, net_name, song_name, midi)
midiUtils.playMidi(midi, save_path + midiUtils.joinNames(net_name, song_name))

Output hidden; open in https://colab.research.google.com to view.

In [0]:
song_name = 'Smells Like Teen Spirit'
seedPath = '/content/gdrive/My Drive/Music Gen/test_generation/test_songs/smells_like_teen_spirit.npz'

midi = modelCompound.generateMidi(
    seedPath,
    280,
    threshold=0.1)

midiUtils.plotMidi(save_path, net_name, song_name, midi)
midiUtils.playMidi(midi, save_path + midiUtils.joinNames(net_name, song_name))

Output hidden; open in https://colab.research.google.com to view.

In [0]:
song_name = 'Under The Bridge'
seedPath = '/content/gdrive/My Drive/Music Gen/test_generation/test_songs/under_the_bridge.npz'

midi = modelCompound.generateMidi(
    seedPath,
    280,
    threshold=0.1)

midiUtils.plotMidi(save_path, net_name, song_name, midi)
midiUtils.playMidi(midi, save_path + midiUtils.joinNames(net_name, song_name))

Output hidden; open in https://colab.research.google.com to view.

In [0]:
song_name = 'Unforgiven 2'
seedPath = '/content/gdrive/My Drive/Music Gen/test_generation/test_songs/unforgiven_2.npz'

midi = modelCompound.generateMidi(
    seedPath,
    280,
    threshold=0.1)

midiUtils.plotMidi(save_path, net_name, song_name, midi)
midiUtils.playMidi(midi, save_path + midiUtils.joinNames(net_name, song_name))

Output hidden; open in https://colab.research.google.com to view.

In [0]:
song_name = 'Unforgiven'
seedPath = '/content/gdrive/My Drive/Music Gen/test_generation/test_songs/unforgiven.npz'

midi = modelCompound.generateMidi(
    seedPath,
    280,
    threshold=0.1)

midiUtils.plotMidi(save_path, net_name, song_name, midi)
midiUtils.playMidi(midi, save_path + midiUtils.joinNames(net_name, song_name))

Output hidden; open in https://colab.research.google.com to view.

In [0]:
song_name = 'Wonderwall'
seedPath = '/content/gdrive/My Drive/Music Gen/test_generation/test_songs/wonderwall.npz'

midi = modelCompound.generateMidi(
    seedPath,
    280,
    threshold=0.1)

midiUtils.plotMidi(save_path, net_name, song_name, midi)
midiUtils.playMidi(midi, save_path + midiUtils.joinNames(net_name, song_name))

Output hidden; open in https://colab.research.google.com to view.